In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
import sys
sys.path.append('/content/drive/MyDrive/Project/models') # append the directory containing the module

In [4]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import EncoderEEG
import deepcnn
import cnn2dlstm
import cnn1dlstm
import traintest

In [30]:
import importlib
importlib.reload(traintest) # reload the module

<module 'traintest' from '/content/drive/MyDrive/Project/models/traintest.py'>

In [5]:
torch.cuda.is_available()
if torch.cuda.is_available():
    print("GPU Name:", torch.cuda.get_device_name(0))

GPU Name: Tesla T4


In [6]:
dataset=torch.load(r"/content/drive/MyDrive/Project/eeg_55_95_std.pth")
labels=dataset['labels']
print(labels)

<ipython-input-6-c1ae3ad9eb8b>:1: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  dataset=torch.load(r"/content/drive/MyDrive/Project/eeg_55_95_std.pth")


['n02389026', 'n03888257', 'n03584829', 'n02607072', 'n03297495', 'n03063599', 'n03792782', 'n04086273', 'n02510455', 'n11939491', 'n02951358', 'n02281787', 'n02106662', 'n04120489', 'n03590841', 'n02992529', 'n03445777', 'n03180011', 'n02906734', 'n07873807', 'n03773504', 'n02492035', 'n03982430', 'n03709823', 'n03100240', 'n03376595', 'n03877472', 'n03775071', 'n03272010', 'n04069434', 'n03452741', 'n03792972', 'n07753592', 'n13054560', 'n03197337', 'n02504458', 'n02690373', 'n03272562', 'n04044716', 'n02124075']


In [7]:
data=dataset['dataset']

In [8]:
splits=torch.load(r"/content/drive/MyDrive/Project/block_splits_by_image_all.pth")

<ipython-input-8-418aa34d7d22>:1: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  splits=torch.load(r"/content/drive/MyDrive/Project/block_splits_by_image_all.pth")


In [9]:
x_train,x_val,x_test,y_train,y_val,y_test=traintest.Splitter(data,splits,subject_no=0)

In [10]:
# For using 2D CNN or similar models, we need to add an extra dimension to our data for which we can use unsqueeze function

x_train=torch.stack(x_train)
#x_train=x_train.unsqueeze(3)
x_val=torch.stack(x_val)
#x_val=x_val.unsqueeze(3)
x_test=torch.stack(x_test)
#x_test=x_test.unsqueeze(3)
y_train=torch.tensor(y_train)
y_val=torch.tensor(y_val)
y_test=torch.tensor(y_test)

In [11]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [12]:
from torch.utils.data import DataLoader, TensorDataset

train_dataset = TensorDataset(x_train, y_train)
val_dataset = TensorDataset(x_val,y_val)
test_dataset = TensorDataset(x_test,y_test)
train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)
val_loader = DataLoader(val_dataset,batch_size=16,shuffle=True)
test_loader = DataLoader(test_dataset,batch_size=16,shuffle=True)

In [ ]:
num_classes = 40
learning_rate = 0.005
num_epochs = 50

model = EncoderEEG.EEGEncoder(num_classes=40).to(device)
print(model)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

EEGEncoder(
  (temporal_block): TemporalBlock(
    (network): Sequential(
      (0): Conv1d(128, 64, kernel_size=(3,), stride=(1,), padding=(3,))
      (1): ReLU()
      (2): Conv1d(64, 64, kernel_size=(3,), stride=(1,), padding=(6,), dilation=(2,))
      (3): ReLU()
      (4): Conv1d(64, 64, kernel_size=(3,), stride=(1,), padding=(12,), dilation=(4,))
      (5): ReLU()
      (6): Conv1d(64, 64, kernel_size=(3,), stride=(1,), padding=(24,), dilation=(8,))
      (7): ReLU()
    )
  )
  (spatial_block): SpatialBlock(
    (network): Sequential(
      (0): Conv2d(1, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): ReLU()
      (3): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (4): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (5): ReLU()
    )
  )
  (res_blocks): ModuleList(
    (0-1): 2 x ResidualBlock(
      (conv1)

In [ ]:
#for inputs,labels in train_loader:
 #   print(inputs.shape)

In [ ]:
# Call training function
traintest.train_model(model, train_loader, val_loader,criterion,optimizer,num_epochs)

Epoch 1/50, Training Loss: 3.6681, Validation Loss: 3.6328, Training Accuracy: 3.14%, Validation Accuracy: 2.40%
Epoch 2/50, Training Loss: 3.5224, Validation Loss: 3.4731, Training Accuracy: 4.09%, Validation Accuracy: 4.30%
Epoch 3/50, Training Loss: 3.2692, Validation Loss: 3.1000, Training Accuracy: 5.50%, Validation Accuracy: 7.56%
Epoch 4/50, Training Loss: 2.9927, Validation Loss: 3.1904, Training Accuracy: 7.69%, Validation Accuracy: 6.16%
Epoch 5/50, Training Loss: 2.7882, Validation Loss: 2.6539, Training Accuracy: 9.79%, Validation Accuracy: 12.06%
Epoch 6/50, Training Loss: 2.6276, Validation Loss: 2.5618, Training Accuracy: 11.04%, Validation Accuracy: 11.21%
Epoch 7/50, Training Loss: 2.5078, Validation Loss: 2.5470, Training Accuracy: 13.96%, Validation Accuracy: 14.66%
Epoch 8/50, Training Loss: 2.4525, Validation Loss: 2.3131, Training Accuracy: 13.96%, Validation Accuracy: 15.82%
Epoch 9/50, Training Loss: 2.3989, Validation Loss: 2.3749, Training Accuracy: 14.69%, Va

In [ ]:
traintest.evaluate_model(model, test_loader)

Test Accuracy: 40.16%


In [ ]:
#Saving the model

torch.save(model.state_dict(), 'EEGEncoder_state_dict.pth')  # Save only the state dict
torch.save(model, 'EEGEncoder.pth')  # Save the entire model
torch.save(optimizer.state_dict(), 'optimizer_state_dict.pth')

##Further Training for a few more epochs

In [17]:
model = torch.load('/content/drive/MyDrive/Project/EEGEncoder.pth', map_location=device)
model.load_state_dict(torch.load('/content/drive/MyDrive/Project/EEGEncoder_state_dict.pth',  map_location=device))
optimizer = torch.optim.Adam(model.parameters(), lr=0.0025)
optimizer.load_state_dict(torch.load('/content/drive/MyDrive/Project/optimizer_state_dict.pth'))
criterion = nn.CrossEntropyLoss()

<ipython-input-17-66adb7acfd41>:1: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model = torch.load('/content/drive/MyDrive/Project/EEGEncoder.pth', map_location=device)
<ip

In [18]:
traintest.train_model(model, train_loader, val_loader,criterion,optimizer,device,num_epochs=50)

Epoch 1/50, Training Loss: 1.2439, Validation Loss: 1.5288, Training Accuracy: 48.01%, Validation Accuracy: 39.99%
Epoch 2/50, Training Loss: 1.2098, Validation Loss: 1.6303, Training Accuracy: 49.72%, Validation Accuracy: 37.54%
Epoch 3/50, Training Loss: 1.2020, Validation Loss: 1.4892, Training Accuracy: 49.99%, Validation Accuracy: 43.29%
Epoch 4/50, Training Loss: 1.1870, Validation Loss: 1.6133, Training Accuracy: 50.94%, Validation Accuracy: 39.64%
Epoch 5/50, Training Loss: 1.2102, Validation Loss: 1.7851, Training Accuracy: 50.26%, Validation Accuracy: 33.78%
Epoch 6/50, Training Loss: 1.2077, Validation Loss: 1.5425, Training Accuracy: 50.95%, Validation Accuracy: 41.94%
Epoch 7/50, Training Loss: 1.1361, Validation Loss: 1.6269, Training Accuracy: 52.66%, Validation Accuracy: 38.79%
Epoch 8/50, Training Loss: 1.1485, Validation Loss: 1.5961, Training Accuracy: 52.23%, Validation Accuracy: 40.09%
Epoch 9/50, Training Loss: 1.1455, Validation Loss: 1.5102, Training Accuracy: 5

In [19]:
traintest.evaluate_model(model, test_loader, device)

Test Accuracy: 44.82%


In [21]:
#Saving the model

torch.save(model.state_dict(), 'EEGEncoder_state_dict2.pth')  # Save only the state dict
torch.save(model, 'EEGEncoder2.pth')  # Save the entire model
torch.save(optimizer.state_dict(), 'optimizer_state_dict2.pth')

## Not Relevant

In [ ]:
sample_input = torch.randn(16, 128, 440)  # Create a sample input tensor

# Initialize your model
num_classes = 40  # Adjust based on the number of classes in your task
model =EncoderEEG.EEGEncoder(num_classes)  # Use the correct model name here

# Pass the sample input through the model
try:
    output = model(sample_input)
    print("Output shape:", output.shape)  # Print the output shape
except Exception as e:
    print("An error occurred:", e)

An error occurred: Given groups=1, weight of size [64, 40, 3], expected input[16, 128, 440] to have 40 channels, but got 128 channels instead


In [ ]:
print(model.parameters)

<bound method Module.parameters of EEGEncoder(
  (temporal_block): TemporalBlock(
    (network): Sequential(
      (0): Conv1d(128, 64, kernel_size=(3,), stride=(1,), padding=(3,))
      (1): ReLU()
      (2): Conv1d(64, 64, kernel_size=(3,), stride=(1,), padding=(6,), dilation=(2,))
      (3): ReLU()
      (4): Conv1d(64, 64, kernel_size=(3,), stride=(1,), padding=(12,), dilation=(4,))
      (5): ReLU()
      (6): Conv1d(64, 64, kernel_size=(3,), stride=(1,), padding=(24,), dilation=(8,))
      (7): ReLU()
    )
  )
  (spatial_block): SpatialBlock(
    (network): Sequential(
      (0): Conv2d(1, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): ReLU()
      (3): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (4): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (5): ReLU()
    )
  )
  (res_blocks): ModuleList(
    (0-1